In [ ]:
import os
import numpy as np
import pandas as pd

# Load your existing iemocap DataFrame
df_iemocap = pd.read_csv('df_iemocap_with_features.csv')  # Adjust the path to your CSV


In [ ]:
df_iemocap.head()

,start_time,end_time,wav_file,emotion,val,act,dom,MFCC1,MFCC2,MFCC3,...,Chroma11,Chroma12,SpectralContrast1,SpectralContrast2,SpectralContrast3,SpectralContrast4,SpectralContrast5,SpectralContrast6,SpectralContrast7,ZeroCrossingRate
0,8.0341,13.50,Ses05F_script01_1_F000,sad,2.5,2.5,2.5,-464.898987,119.969124,-7.643576,...,0.421615,0.375193,17.053972,16.371297,16.881244,17.185019,17.721546,17.477850,20.656050,0.078382
1,15.2259,19.10,Ses05F_script01_1_F001,xxx,2.0,3.0,3.0,-435.348572,117.497269,-10.906404,...,0.380136,0.421472,17.265016,16.791738,17.651642,17.205775,17.052922,17.942451,21.527245,0.094663
2,20.5700,22.10,Ses05F_script01_1_F002,xxx,2.5,2.0,2.5,-498.951324,135.751755,-14.035595,...,0.631194,0.421012,15.005613,14.258981,16.267633,17.036546,17.999499,17.367494,17.658056,0.060699
3,28.9800,32.13,Ses05F_script01_1_F003,xxx,2.5,3.0,2.5,-459.131958,127.234604,-10.658073,...,0.432782,0.388997,15.970424,17.969763,17.118443,17.177244,17.301360,18.076253,19.646218,0.064700
4,38.1100,39.88,Ses05F_script01_1_F004,sad,2.5,2.5,3.0,-472.654846,118.265526,-8.783584,...,0.453893,0.584195,15.275234,16.910328,16.387280,16.713556,17.352526,18.122145,19.914245,0.084891


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# pip install tensorflow

In [ ]:
df_iemocap = df_iemocap.dropna()

In [ ]:
emotion_dict = {'ang': 0,
                'hap': 1,
                'exc': 2,
                'sad': 3,
                'fru': 4,
                'fea': 5,
                'sur': 6,
                'neu': 7,
                'xxx': 8,
                'oth': 8,
                'dis': 8}

In [ ]:
df_iemocap['emotion_label'] = df_iemocap['emotion'].map(emotion_dict)


In [ ]:
df_iemocap['emotion_label'] = df_iemocap['emotion_label'].astype('float64')


In [ ]:
print(df_iemocap[['emotion', 'emotion_label']].head())


  emotion  emotion_label
0     sad            3.0
1     xxx            8.0
2     xxx            8.0
3     xxx            8.0
4     sad            3.0


In [ ]:
# Separate features and targets
X = df_iemocap.drop(columns=['emotion','emotion_label', 'val', 'act', 'dom', 'start_time', 'end_time', 'wav_file'])
y = df_iemocap[['emotion_label', 'val', 'act', 'dom']]
y_emotion = df_iemocap['emotion_label']
y_val = df_iemocap['val']
y_act = df_iemocap['act']
y_dom = df_iemocap['dom']

In [ ]:
# Normalize the features (X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train_emotion, y_test_emotion = train_test_split(X_scaled, y_emotion, test_size=0.2, random_state=42)


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Input
from sklearn.preprocessing import LabelEncoder


In [ ]:
print(tf.__version__)
print(np.__version__)

2.17.0
1.26.4


In [ ]:
print(X_train.shape)  # This should print (number_of_samples, 1, number_of_features)


(1736, 33)


In [ ]:
# Assuming X_train is a 2D array (2170, 33)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # Reshape to (samples, timesteps, features)


In [ ]:
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(9, activation='softmax'))  # Output layer with softmax for multi-class


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
history = model.fit(X_train, y_train_emotion, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.1940 - loss: 2.1491 - val_accuracy: 0.2816 - val_loss: 1.9776
Epoch 2/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3710 - loss: 1.9087 - val_accuracy: 0.3132 - val_loss: 1.8394
Epoch 3/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3770 - loss: 1.7845 - val_accuracy: 0.3190 - val_loss: 1.7604
Epoch 4/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3609 - loss: 1.7511 - val_accuracy: 0.3017 - val_loss: 1.7151
Epoch 5/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3827 - loss: 1.6710 - val_accuracy: 0.3276 - val_loss: 1.6905
Epoch 6/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3829 - loss: 1.6512 - val_accuracy: 0.3420 - val_loss: 1.6671
Epoch 7/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3783 - loss: 1.6453 - val_accuracy: 0.3276 - val_loss: 1.6530
Epoch 8/50
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4182 - loss: 1.5843 - val_accuracy: 0.3477 - val_loss

In [ ]:
print(X_test.shape)  # Should be (num_samples, timesteps, num_features)


(434, 33)


In [ ]:
X_test_reshaped = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


In [ ]:
loss, accuracy = model.evaluate(X_test_reshaped, y_test_emotion)  # Use the correctly formatted labels
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3982 - loss: 1.5708 
Test Loss: 1.6227, Test Accuracy: 0.3687
